# Copyright

<PRE>
Jelen iPython notebook a Budapesti Műszaki és Gazdaságtudományi Egyetemen tartott 
"Deep Learning a gyakorlatban Python és LUA alapon" tantárgy segédanyagaként készült.

A tantárgy honlapja: http://smartlab.tmit.bme.hu/oktatas-deep-learning 
Deep Learning kutatás: http://smartlab.tmit.bme.hu/deep-learning

A forráskódot GPLv3 licensz védi. Újrafelhasználás esetén lehetőség szerint kérjük az alábbi szerzőt értesíteni.

2016 (c) Szaszák György (szaszak kukac tmit pont bme pont hu)
</PRE>


In [1]:
import numpy as np
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# random seed a könnyebb reprodukálhatósághoz
seed = 77
np.random.seed(seed)

Using Theano backend.


## Sentiment elemzés (tetszik/nem tetszik) szóbeágyazás alapján filmkritikákból

In [7]:
# Betöltjük az adatokat (Keras támogatott)
top_words=6000 # Limitáljuk a szavak számát, 6000 leggyakoribbat hagyjuk meg, többit kidobjuk
skip_top=20 # Abból is kihagyhatjuk mondjuk a 20 leggyakoribbat -> érdemes így is futtatni és összehasonlítani az eredményt

(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=top_words,skip_top=skip_top)

# Nincs validációs halmaz, lecsípünk a teszthalmazból validációhoz
X_val=X_test[:15000]
X_test=X_test[15000:]
y_val=y_test[:15000]
y_test=y_test[15000:]

# Adatok alakja, osztályok száma
print("Adataink: ")
print(X_train.shape)
print(y_train.shape)

print(y_val.shape)
print(y_test.shape)

print("Osztályok: ")
print(np.unique(y_train))

print("Szavak száma: ")
print(len(np.unique(np.hstack(X_train))))

print("Egy minta: ")
print(X_val[0])
print(y_val[0])

Adataink: 
(25000,)
(25000,)
(15000,)
(10000,)
Osztályok: 
[0 1]
Szavak száma: 
5981
Egy minta: 
[2, 89, 27, 2, 2, 2, 199, 132, 2, 4191, 2, 1339, 24, 2, 760, 2, 1385, 2, 2, 22, 1368, 2, 2, 5149, 2, 1635, 2, 2, 1368, 2, 2, 1357, 2, 2, 991, 2, 877, 38, 2, 27, 239, 2, 100, 235, 61, 483, 2, 2, 2, 2, 20, 131, 1102, 72, 2, 2, 251, 27, 1146, 2, 308, 2, 735, 1517, 2, 29, 144, 28, 77, 2305, 2, 2]
1


## Padding egyenlő hosszra

In [8]:
# Ennyi szót fogunk kritikánként figyelembe venni, ha ennél rövidebb kritika van, azt nullákkal kipaddingeli a pad_sequences().
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_val = sequence.pad_sequences(X_val, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

print("Első minta padding után")
print(X_train[0])

Első minta padding után
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    

## Modell összeállítása, tanítás és tesztelés

In [9]:
embedding_dim=50

model = Sequential()
# Mi csináljuk az embeddinget is
model.add(Embedding(top_words, embedding_dim, input_length=max_words))
#model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_2 (Embedding)          (None, 500, 50)       300000      embedding_input_2[0][0]          
____________________________________________________________________________________________________
flatten_2 (Flatten)              (None, 25000)         0           embedding_2[0][0]                
____________________________________________________________________________________________________
dense_3 (Dense)                  (None, 250)           6250250     flatten_2[0][0]                  
____________________________________________________________________________________________________
dense_4 (Dense)                  (None, 1)             251         dense_3[0][0]                    
Total params: 6550501
_____________________________________________________________________

In [10]:
# Tanítás (csak 2 epoch, nincs túltanulás elleni védelmünk)
model.fit(X_train, y_train, validation_data=(X_val, y_val), nb_epoch=2, batch_size=128, verbose=1)

Train on 25000 samples, validate on 15000 samples
Epoch 1/2
25000/25000 [==============================] - 27s - loss: 0.4676 - acc: 0.7529 - val_loss: 0.2823 - val_acc: 0.8813
Epoch 2/2
25000/25000 [==============================] - 27s - loss: 0.1632 - acc: 0.9395 - val_loss: 0.3081 - val_acc: 0.8763


In [11]:
# Kiértékelés a tesztadatokon
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 87.07%
